In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# This source code is licensed under the BSD-style license found in the
# LICENSE file in the root directory of this source tree.

In [ ]:
from src import LTVSyntheticData
from src import LTVexploratory

### Generate synthetic data 

In [ ]:
synth_data_gen = LTVSyntheticData(is_subscr=False,              
                                  share_payers=0.05,
                                  share_high_payers= 0.05,
                                  share_low_payers= 0.8)
ancor_table = synth_data_gen.get_ancor_table()
event_table = synth_data_gen.get_purchases()

# Call the exploratory class
da = LTVexploratory(ancor_table, event_table)

### Run analysis

In [ ]:
da.plot_n()

In [ ]:
da.plot_first_purchase()

In [ ]:
da.plot_second_purchase()

In [ ]:
da.plot_uuid()

In [ ]:
da.plot_linear_corr()

In [ ]:
# Not working! 
#da.plot_sankey()

In [ ]:
da.plot_class_change()

In [ ]:
da.plot_drop_off()

In [ ]:
# Needs better name! 
da.plot_()

In [ ]:
da.plot_freq()

In [ ]:
da.plot_apv()

In [ ]:
da.plot_corr_freq_apv()

In [ ]:
da.plot_users_intersection()

In [ ]:
import numpy as np

self = da
days_limit = 60
granularity =1000
# Select only users that are at least [days_limit] days old
end_events_date = self.joined_df[self.event_time_col].max()
data = self.joined_df[
    (end_events_date - self.joined_df[self.registration_time_col]).dt.days
    >= days_limit
].copy()

# Remove users who never had a purchase and ensure all users have the same opportunity window
data = data[
    (data[self.event_time_col] - data[self.registration_time_col]).dt.days
    <= days_limit
]

# Count how many purchase users had (defined by value > 0) and then how many users are in each place
data = data[data[self.value_col] > 0]
data = (
    data.groupby(self.uuid_col)[self.value_col].sum()
    .reset_index()
    .sort_values(self.value_col, ascending=False)
)

# Calculate total revenue and group users together based on the granilarity
total_revenue = data[self.value_col].sum()
total_users = data.shape[0]
data['cshare_users'] = [(i+1)/total_users for i in range(total_users)]
data['cshare_revenue'] = data[self.value_col].cumsum() / total_revenue
data.head(30)
# data['group'] = data['cshare_users'].apply(lambda x: np.ceil(x*granularity))
# data = (
#     data
#     .groupby("group")[["cshare_users", "cshare_revenue"]]
#     .max()
#     .reset_index()
# )

# self.graph.line_plot(
#             data,
#             x_axis="cshare_users",
#             y_axis="cshare_revenue",
#             xlabel="Share of paying customers",
#             ylabel="Share of revenue",
#             x_format="%",
#             y_format="%",
#             title=f"Share of all revenue of the first {days_limit} days after customer registration (Y) versus share of paying customers",
#         )

In [ ]:
da.plot_revenue_pareto(days_limit=60)

In [ ]:
da.plot_paying_users_flow(days_limit=60, early_limit=7, spending_breaks={'No spend': 0, 'Low spend': 10, 'Medium spend': 100, 'High spend': 99999})

In [ ]:
da.plot_user_histogram_per_conversion_day(days_limit=60)

In [ ]:
da.plot_early_late_revenue_correlation(days_limit=70)
